In [1]:

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
train_df_org = pd.read_csv('train.csv')
test_df_org = pd.read_csv('test.csv')
train_df=train_df_org.copy()
test_df = test_df_org.copy()

combine_org = [train_df_org, test_df_org]
combine = [train_df, test_df]

**컬럼명확인**

**트레인 데이터는 중식계 석식계 있음 피처 12개**

In [3]:
print("컬럼명확인\n",train_df.columns.values)
print(train_df.shape)
train_df.head()

컬럼명확인
 ['일자' '요일' '본사정원수' '본사휴가자수' '본사출장자수' '본사시간외근무명령서승인건수' '현본사소속재택근무자수' '조식메뉴'
 '중식메뉴' '석식메뉴' '중식계' '석식계']
(1205, 12)


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [4]:
for dataset in combine:
    dataset.drop(['조식메뉴'],axis=1, inplace=True) 




**세미 홀리데이 만들기**

기준 1. 석식계가 0인날(train 데이터에만 적용)
(test 데이터는 문화의 날 없어졌으니 이 데이터 빼고 학습)

In [5]:

# print(train_df[train_df['석식계']==0])
idx_cultureday = train_df[train_df['석식계']==0].index
print(idx_cultureday)
train_df.drop(idx_cultureday,inplace=True)
print(train_df.shape)

Int64Index([ 204,  224,  244,  262,  281,  306,  327,  346,  366,  392,  410,
             412,  424,  449,  468,  492,  502,  510,  529,  549,  571,  589,
             609,  633,  648,  667,  687,  706,  730,  747,  766,  786,  809,
             828,  853,  872,  890,  912,  932,  955,  973,  993, 1166],
           dtype='int64')
(1162, 11)


휴가자수가 과도하게 많은날을 semiholiday 로 지정(피처 새로 만들기)

In [6]:
#휴가자수가 과도하게 많은날을 semiholiday로 지정
nintynine_percentile = train_df['본사휴가자수'].quantile(q=0.99, interpolation='nearest')
for dataset in combine:
    dataset['semiholiday'] = 0
    dataset.loc[dataset['본사휴가자수']>nintynine_percentile,'semiholiday']=1
  

휴가자나 출장자는 점심 저녁중에 하나는 먹을 수도 있음. 따라서, 총 정원에서 재택근무자만 뺀걸로 새로운 피처 만들기

In [7]:
for dataset in combine:
    dataset['실근무자']= dataset['본사정원수']-dataset['현본사소속재택근무자수']
  

재택근무자수 10을 기준으로 애프터 코로나 기준 만들기

In [8]:
for dataset in combine:
    dataset['aftercorona'] = 0
    dataset.loc[dataset['현본사소속재택근무자수']>10, 'aftercorona'] =1

본사 정원 현본사 재택근무자수 피처 삭제

In [9]:
for dataset in combine:
    dataset.drop(['본사정원수','현본사소속재택근무자수'],axis=1, inplace=True)

요일 원핫 인코딩

In [10]:
sample1=pd.DataFrame(train_df['요일'])
daycolumn1=pd.get_dummies(sample1)
train_df1 = pd.concat([train_df,daycolumn1],axis=1)
train_df= train_df1

In [11]:
sample2=pd.DataFrame(test_df['요일'])
daycolumn2=pd.get_dummies(sample2)
test_df2 = pd.concat([test_df,daycolumn2],axis=1)
test_df= test_df2

In [12]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 15 columns):
일자                50 non-null object
요일                50 non-null object
본사휴가자수            50 non-null int64
본사출장자수            50 non-null int64
본사시간외근무명령서승인건수    50 non-null int64
중식메뉴              50 non-null object
석식메뉴              50 non-null object
semiholiday       50 non-null int64
실근무자              50 non-null float64
aftercorona       50 non-null int64
요일_금              50 non-null uint8
요일_목              50 non-null uint8
요일_수              50 non-null uint8
요일_월              50 non-null uint8
요일_화              50 non-null uint8
dtypes: float64(1), int64(5), object(4), uint8(5)
memory usage: 4.3+ KB


요일 피처 삭제

In [13]:
train_df.drop(['요일'],axis=1, inplace=True)
test_df.drop(['요일'],axis=1, inplace=True)

In [14]:
train_df=train_df.rename(columns={'일자':'datetime'})
test_df= test_df.rename(columns={'일자':'datetime'})

월 데이터 추출 후 일자 데이터 삭제

In [15]:
train_df['datetime'] = train_df.datetime.apply(pd.to_datetime)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1162 entries, 0 to 1204
Data columns (total 16 columns):
datetime          1162 non-null datetime64[ns]
본사휴가자수            1162 non-null int64
본사출장자수            1162 non-null int64
본사시간외근무명령서승인건수    1162 non-null int64
중식메뉴              1162 non-null object
석식메뉴              1162 non-null object
중식계               1162 non-null float64
석식계               1162 non-null float64
semiholiday       1162 non-null int64
실근무자              1162 non-null float64
aftercorona       1162 non-null int64
요일_금              1162 non-null uint8
요일_목              1162 non-null uint8
요일_수              1162 non-null uint8
요일_월              1162 non-null uint8
요일_화              1162 non-null uint8
dtypes: datetime64[ns](1), float64(3), int64(5), object(2), uint8(5)
memory usage: 114.6+ KB


In [16]:
train_df['month'] = train_df.datetime.apply(lambda x : x.month)
print(train_df.info())
train_df.head(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1162 entries, 0 to 1204
Data columns (total 17 columns):
datetime          1162 non-null datetime64[ns]
본사휴가자수            1162 non-null int64
본사출장자수            1162 non-null int64
본사시간외근무명령서승인건수    1162 non-null int64
중식메뉴              1162 non-null object
석식메뉴              1162 non-null object
중식계               1162 non-null float64
석식계               1162 non-null float64
semiholiday       1162 non-null int64
실근무자              1162 non-null float64
aftercorona       1162 non-null int64
요일_금              1162 non-null uint8
요일_목              1162 non-null uint8
요일_수              1162 non-null uint8
요일_월              1162 non-null uint8
요일_화              1162 non-null uint8
month             1162 non-null int64
dtypes: datetime64[ns](1), float64(3), int64(6), object(2), uint8(5)
memory usage: 123.7+ KB
None


,datetime,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,중식메뉴,석식메뉴,중식계,석식계,semiholiday,실근무자,aftercorona,요일_금,요일_목,요일_수,요일_월,요일_화,month
0,2016-02-01,50,150,238,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0,2601.0,0,0,0,0,1,0,2
1,2016-02-02,50,173,319,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0,2601.0,0,0,0,0,0,1,2
2,2016-02-03,56,180,111,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0,2601.0,0,0,0,1,0,0,2


In [17]:
test_df['datetime'] = test_df.datetime.apply(pd.to_datetime)
test_df['month'] = test_df.datetime.apply(lambda x : x.month)
print(test_df.info())
train_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 15 columns):
datetime          50 non-null datetime64[ns]
본사휴가자수            50 non-null int64
본사출장자수            50 non-null int64
본사시간외근무명령서승인건수    50 non-null int64
중식메뉴              50 non-null object
석식메뉴              50 non-null object
semiholiday       50 non-null int64
실근무자              50 non-null float64
aftercorona       50 non-null int64
요일_금              50 non-null uint8
요일_목              50 non-null uint8
요일_수              50 non-null uint8
요일_월              50 non-null uint8
요일_화              50 non-null uint8
month             50 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(6), object(2), uint8(5)
memory usage: 4.3+ KB
None


,datetime,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,중식메뉴,석식메뉴,중식계,석식계,semiholiday,실근무자,aftercorona,요일_금,요일_목,요일_수,요일_월,요일_화,month
0,2016-02-01,50,150,238,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0,2601.0,0,0,0,0,1,0,2
1,2016-02-02,50,173,319,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0,2601.0,0,0,0,0,0,1,2
2,2016-02-03,56,180,111,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,0,2601.0,0,0,0,1,0,0,2


In [18]:
train_df.drop(['datetime'],axis=1, inplace=True)
test_df.drop(['datetime'],axis=1, inplace=True)



In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1162 entries, 0 to 1204
Data columns (total 16 columns):
본사휴가자수            1162 non-null int64
본사출장자수            1162 non-null int64
본사시간외근무명령서승인건수    1162 non-null int64
중식메뉴              1162 non-null object
석식메뉴              1162 non-null object
중식계               1162 non-null float64
석식계               1162 non-null float64
semiholiday       1162 non-null int64
실근무자              1162 non-null float64
aftercorona       1162 non-null int64
요일_금              1162 non-null uint8
요일_목              1162 non-null uint8
요일_수              1162 non-null uint8
요일_월              1162 non-null uint8
요일_화              1162 non-null uint8
month             1162 non-null int64
dtypes: float64(3), int64(6), object(2), uint8(5)
memory usage: 114.6+ KB


메뉴 데이터 처리
점심,저녁 train
점심 저녁 test순

점심 저녁 train 데이터

In [20]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch) 만들기 
lunch = []
dinner = []
for day in range(len(train_df)):
    tmp_lun = train_df.iloc[day, 3].split(' ') # 공백으로 문자열 구분 
    tmp_lun = ' '.join(tmp_lun).split()    # 빈 원소 삭제
    tmp_din = train_df.iloc[day, 4].split(' ') # 공백으로 문자열 구분 
    tmp_din = ' '.join(tmp_din).split()    # 빈 원소 삭제
    
    
    search = '('   # 원산지 정보는 삭제
    for menu in tmp_lun:
        if search in menu:
            tmp_lun.remove(menu)
    lunch.append(tmp_lun) 
    
    search = '('   # 원산지 정보는 삭제
    for menu in tmp_din:
        if search in menu:
            tmp_din.remove(menu)
    dinner.append(tmp_din) 

In [21]:
print(lunch[1025])
print(dinner[0:5])

['쌀밥/잡곡밥', '냉모밀국수', '매운돈갈비찜', '메밀전병*간장', '고구마순볶음', '포기김치', '양상추샐러드*딸기요거트']
[['쌀밥/잡곡밥', '육개장', '자반고등어구이', '두부조림', '건파래무침', '포기김치'], ['콩나물밥*양념장', '어묵국', '유산슬', '아삭고추무침', '바나나', '포기김치'], ['쌀밥/잡곡밥', '청국장찌개', '황태양념구이', '고기전', '새송이버섯볶음', '포기김치'], ['미니김밥*겨자장', '우동', '멕시칸샐러드', '군고구마', '무피클', '포기김치'], ['쌀밥/잡곡밥', '차돌박이찌개', '닭갈비', '감자소세지볶음', '콩나물무침', '포기김치']]


In [22]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
본사휴가자수            50 non-null int64
본사출장자수            50 non-null int64
본사시간외근무명령서승인건수    50 non-null int64
중식메뉴              50 non-null object
석식메뉴              50 non-null object
semiholiday       50 non-null int64
실근무자              50 non-null float64
aftercorona       50 non-null int64
요일_금              50 non-null uint8
요일_목              50 non-null uint8
요일_수              50 non-null uint8
요일_월              50 non-null uint8
요일_화              50 non-null uint8
month             50 non-null int64
dtypes: float64(1), int64(6), object(2), uint8(5)
memory usage: 3.9+ KB


In [23]:
# 일별 점심메뉴를 작은 리스트로 갖고 있는 큰 리스트 (lunch) 만들기 
lunch_test = []
dinner_test = []
for day in range(len(test_df)):
    tmp_lun = test_df.iloc[day, 3].split(' ') # 공백으로 문자열 구분 
    tmp_lun = ' '.join(tmp_lun).split()    # 빈 원소 삭제
    tmp_din = test_df.iloc[day, 4].split(' ') # 공백으로 문자열 구분 
    tmp_din = ' '.join(tmp_din).split()    # 빈 원소 삭제
    
    
    search = '('   # 원산지 정보는 삭제
    for menu in tmp_lun:
        if search in menu:
            tmp_lun.remove(menu)
    lunch_test.append(tmp_lun) 
    
    search = '('   # 원산지 정보는 삭제
    for menu in tmp_din:
        if search in menu:
            tmp_din.remove(menu)
    dinner_test.append(tmp_din) 

In [24]:
print(lunch_test[0:5])

[['쌀밥/흑미밥/찰현미밥', '대구지리', '매운돈갈비찜', '오꼬노미계란말이', '상추무침', '포기김치', '양상추샐러드*딸기D'], ['쌀밥/보리밥/찰현미밥', '우렁된장찌개', '오리주물럭', '청양부추전', '수제삼색무쌈', '겉절이김치', '양상추샐러드*오미자D'], ['쌀밥/흑미밥/찰현미밥', '팽이장국', '수제돈까스*소스', '가자미조림', '동초나물무침', '포기김치', '양상추샐러드*파인요거트D'], ['쌀밥/흑미밥/찰현미밥', '배추들깨국', '오리대패불고기', '시금치프리타타', '부추고추장무침', '포기김치', '양상추샐러드*망고D'], ['쌀밥/팥밥/찰현미밥', '부대찌개', '닭살데리야끼조림', '버섯탕수', '세발나물무침', '알타리김치/사과푸딩', '양상추샐러드*오리엔탈D']]


문자열(띄어쓰기로) 구성된 tf-idf 전 전처리 완료 상태로 만들기

In [61]:
def toalist(time):
    sample= []
    for i in time:
        a = " ".join(i)
        sample.append(a)
    sample2=[]
    for i in sample:
        if i.find("/"):
            i = i.replace("/"," ")
        if i.find("*"):
            i = i.replace("*"," ")
        if i.find("&"):
            i = i.replace("&"," ")
        sample2.append(i)
    return sample2

In [62]:
ln_tr = toalist(lunch)
dn_tr = toalist(dinner)
ln_te = toalist(lunch_test) 
dn_te = toalist(dinner_test)

In [55]:
print(ln_tr.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 1 columns):
0    1162 non-null object
dtypes: object(1)
memory usage: 9.2+ KB
None


In [56]:

ln_tr = [str (item) for item in ln_tr]
ln_te = [str (item) for item in ln_te]

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(ln_tr)
ln_tr_tfidf_vect = tfidf_vect.transform(ln_tr)
ln_te_tfidf_vect = tfidf_vect.transform(ln_te)

print(ln_tr_tfidf_vect.shape,ln_te_tfidf_vect.shape )

(1162, 1579) (50, 1579)


In [65]:
tfidf_vect2 = TfidfVectorizer()
tfidf_vect2.fit(dn_tr)
dn_tr_tfidf_vect = tfidf_vect2.transform(dn_tr)
dn_te_tfidf_vect = tfidf_vect2.transform(dn_te)

print(dn_tr_tfidf_vect.shape,dn_te_tfidf_vect.shape )

(1162, 1532) (50, 1532)


In [71]:
ln_tr_tfidf_vect

<1162x1579 sparse matrix of type '<class 'numpy.float64'>'
	with 8826 stored elements in Compressed Sparse Row format>

기본적으로 test데이터랑 train데이터에 메뉴들이 다 담기긴 함.

In [ ]:
#eda해준 코드를 이용해볼까하다가, 디너는 또 다르고, 고려해야할 요인이 많아 변수도 너무 많아서, tf-idf 쓰는게 괜찮을 거라 생각, 어디에나 있는 밥
#이런거는 덜 처리될거고, /랑 *  처리만 다시해서 리스트로 만든담에 걍 tf-idf 돌리는게 쉬울듯 낼하자(쌀밥 이런거 좀 감소될듯)
#그런담에 점심 저녁 트레인 데이터(컬럼명 변경한거에), 점심저녁 테스트 데이터 (컬럼명 변경한거) 만들고
# 그중에 쓸 피처만 골라서 train_ln train_dn test_ln test_dn 만들고, 거기에 각각tf-idf로 만든 피처 추가하고
# 그렇게 된걸로 모델돌리기(선형회귀, lightgbm ,xgbooost 이런걸로 대충돌리고 예측값 도출시키자)


In [72]:
train = train_df.copy()
train.columns = ['dayoff', 'bustrip', 'ovtime', 'ln', 'dn', 'target_ln','target_dn', 'semiholiday', 'networker', 'aftercorona', 'fri', 'thu', 'wed','mon','tue','month']
train.head(2)




,dayoff,bustrip,ovtime,ln,dn,target_ln,target_dn,semiholiday,networker,aftercorona,fri,thu,wed,mon,tue,month
0,50,150,238,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,0,2601.0,0,0,0,0,1,0,2
1,50,173,319,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,0,2601.0,0,0,0,0,0,1,2


In [73]:
test = test_df.copy()
test.columns = ['dayoff', 'bustrip', 'ovtime', 'ln', 'dn', 'semiholiday', 'networker', 'aftercorona', 'fri', 'thu', 'wed','mon','tue','month']
test.head(2)

,dayoff,bustrip,ovtime,ln,dn,semiholiday,networker,aftercorona,fri,thu,wed,mon,tue,month
0,88,182,5,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,0,2625.0,1,0,0,1,0,0,1
1,104,212,409,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,0,2635.0,1,0,1,0,0,0,1


점심 예측 모델 데이터 만들기(tf-idf)

In [74]:
train_ln = train.drop(['ln','dn','target_dn'],axis=1)
test_ln = test.drop(['ln','dn'],axis=1)
train_dn = train.drop(['ln','dn','target_ln'],axis=1)
test_dn = test.drop(['ln','dn'],axis=1)

In [75]:
train_ln.head()

,dayoff,bustrip,ovtime,target_ln,semiholiday,networker,aftercorona,fri,thu,wed,mon,tue,month
0,50,150,238,1039.0,0,2601.0,0,0,0,0,1,0,2
1,50,173,319,867.0,0,2601.0,0,0,0,0,0,1,2
2,56,180,111,1017.0,0,2601.0,0,0,0,1,0,0,2
3,104,220,355,978.0,0,2601.0,0,0,1,0,0,0,2
4,278,181,34,925.0,0,2601.0,0,1,0,0,0,0,2


In [84]:
from scipy import sparse
lntr_y = train_ln['target_ln'].to_numpy()
lntr_x = train_ln.drop(['target_ln'],axis=1).to_numpy()
lnte_x = test_ln.to_numpy()


lntr_y.shape

(1162,)

In [90]:
from scipy import sparse
dntr_y = train_dn['target_dn'].to_numpy()
dntr_x = train_dn.drop(['target_dn'],axis=1).to_numpy()
dnte_x = test_dn.to_numpy()

dntr_y.shape

(1162,)

In [85]:


from scipy.sparse import hstack
sparse_matrix = (lntr_x,ln_tr_tfidf_vect)
lntr_x = hstack(sparse_matrix).tocsr()
print(type(lntr_x),lntr_x.shape)

<class 'scipy.sparse.csr.csr_matrix'> (1162, 1591)


In [86]:
sparse_matrix2 = (lnte_x,ln_te_tfidf_vect)
lnte_x = hstack(sparse_matrix2).tocsr()
print(type(lnte_x),lnte_x.shape)



<class 'scipy.sparse.csr.csr_matrix'> (50, 1591)


In [91]:
from scipy.sparse import hstack
sparse_matrix = (dntr_x,dn_tr_tfidf_vect)
dntr_x = hstack(sparse_matrix).tocsr()
print(type(dntr_x),dntr_x.shape)

<class 'scipy.sparse.csr.csr_matrix'> (1162, 1544)


In [92]:
sparse_matrix2 = (dnte_x,dn_te_tfidf_vect)
dnte_x = hstack(sparse_matrix2).tocsr()
print(type(dnte_x),dnte_x.shape)

<class 'scipy.sparse.csr.csr_matrix'> (50, 1544)


점심예측모델 모델에 데이터 적용

In [87]:
from sklearn.ensemble import RandomForestRegressor
model_ln = RandomForestRegressor(n_jobs=-1, random_state=42)
model_ln.fit(lntr_x, lntr_y)
pred_ln = model_ln.predict(lnte_x)
print(pred_ln)

[1020.2 1029.6  689.7 1210.8 1001.6 1022.2 1023.4  719.1 1291.1  994.1
  840.5 1238.1 1119.  1022.8  941.4  724.4 1220.9 1010.9  961.   887.2
  580.3 1031.1  998.1  972.1  619.8 1258.4 1108.  1036.8  985.5  572.
 1263.9 1000.2  984.1  956.2  710.7 1214.3 1007.4  918.7  914.2  540.1
 1251.9 1072.4 1005.1  866.   538.6 1208.  1062.9  961.5  902.8  555.7]


저녁 예측모델 모델에 데이터 적용

In [93]:
from sklearn.ensemble import RandomForestRegressor
model_dn = RandomForestRegressor(n_jobs=-1, random_state=42)
model_dn.fit(dntr_x, dntr_y)
pred_dn = model_dn.predict(dnte_x)
print(pred_dn)

[420.8 436.8 263.5 487.6 433.  485.2 402.3 450.3 616.2 498.1 279.6 721.
 651.2 450.  503.5 396.5 630.7 616.1 455.4 461.1 251.7 613.1 458.1 550.6
 348.6 635.8 625.5 423.8 590.1 282.  724.4 585.  421.7 418.8 251.  600.6
 637.6 428.9 380.5 276.3 658.3 638.6 449.  422.7 281.2 550.7 589.8 447.1
 442.1 279. ]


제출용 데이터 만들기

In [94]:
submission = pd.read_csv('sample_submission.csv')

In [95]:
submission['중식계'] = pred_ln
submission['석식계'] = pred_dn
submission.to_csv('baseline.csv', index=False)